In [5]:
#---載入模組---
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import cv2
#---載入模組---

!wget https://i.ytimg.com/vi/rfo-S3yGrJE/hqdefault.jpg #載入圖片連結


from skimage import data, color, feature
import skimage.data

image = cv2.imread('hqdefault.jpg', cv2.IMREAD_GRAYSCALE) #讀取圖片並轉成灰階


#---HOG的演算過程---
hog_vec, hog_vis = feature.hog(image, visualise=True) #取得其HOG資訊及視覺化圖檔

fig, ax = plt.subplots(1, 2, figsize=(12, 6), subplot_kw=dict(xticks=[], yticks=[])) #建立1X2的圖像和12X6的畫布
ax[0].imshow(image, cmap='gray') #轉為灰階
ax[0].set_title('input image') #圖片標題

ax[1].imshow(hog_vis) #顯示視覺化圖檔
ax[1].set_title('visualization of HOG features'); #圖片標題
#---HOG的演算過程---


##---下載數據#---
#匯入人臉的資料集作為正面的訓練資料
from sklearn.datasets import fetch_lfw_people 

faces = fetch_lfw_people(min_faces_per_person = 60) #加载lfw人臉辨識資料集的函数
positive_patches = faces.images
positive_patches.shape
#---下載數據#---

#---下載數據---
#匯入不為人臉的資料集作為訓練負面的資料
from skimage import data, transform

imgs_to_use = ['camera', 'text', 'coins', 'moon',
               'page', 'clock', 'immunohistochemistry',
               'chelsea', 'coffee', 'hubble_deep_field']
images = [color.rgb2gray(getattr(data, name)())
          for name in imgs_to_use]
          
from sklearn.feature_extraction.image import PatchExtractor

def extract_patches(img, N, scale=1.0, patch_size=positive_patches[0].shape):
    extracted_patch_size = tuple((scale * np.array(patch_size)).astype(int))
    extractor = PatchExtractor(patch_size=extracted_patch_size,
                               max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch, patch_size)
                            for patch in patches])
    return patches

negative_patches = np.vstack([extract_patches(im, 1000, scale)
                              for im in images for scale in [0.5, 1.0, 2.0]])
negative_patches.shape

#---下載數據---

#---計算HOG的特徵---
#組合正負兩面的樣本合併計算HOG的特徵
from itertools import chain
X_train = np.array([feature.hog(im)
                    for im in chain(positive_patches, negative_patches)])
y_train = np.zeros(X_train.shape[0])
y_train[:positive_patches.shape[0]] = 1
X_train.shape
#---計算HOG的特徵---

#---獲取baesline---
#用一個簡單得高斯貝氏分類器來會獲取baesline
#並利用sklearn中的LinearSVC來更方便的縮放樣本數據的資料
#from sklearn.cross_validation import cross_val_score(廢除)
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

cross_val_score(GaussianNB(), X_train, y_train) #交叉驗證


from sklearn.svm import LinearSVC
#from sklearn.grid_search import GridSearchCV (廢除)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(LinearSVC(), {'C': [1.0, 2.0, 4.0, 8.0]})
grid.fit(X_train, y_train)
grid.best_score_

model = grid.best_estimator_
model.fit(X_train, y_train)
#---獲取baesline---

#---切割多個cell---
#用SKlearn資料集中的圖片為測試資料
#首先要先轉換為灰階
#並且將圖片切割多個cell
test_image = skimage.data.astronaut() #宇航員艾琳柯林斯的彩色圖片
test_image = skimage.color.rgb2gray(test_image) #轉成灰階
test_image = cv2.imread('hqdefault.jpg', cv2.IMREAD_GRAYSCALE) #轉成灰階
test_image = skimage.transform.rescale(test_image, 0.5)#放大圖片
test_image = test_image[:160, 40:180]

plt.imshow(test_image, cmap='gray') #以灰階顯示圖片
plt.axis('off');
#---切割多個cell---


#---運算每個cell的特徵值---
#跑過一張圖片的所有cell來運算每個cell的特徵值
def sliding_window(img, patch_size=positive_patches[0].shape,
                   istep=2, jstep=2, scale=1.0):
    Ni, Nj = (int(scale * s) for s in patch_size)
    for i in range(0, img.shape[0] - Ni, istep):
        for j in range(0, img.shape[1] - Ni, jstep):
            patch = img[i:i + Ni, j:j + Nj]
            if scale != 1:
                patch = transform.resize(patch, patch_size)
            yield (i, j), patch
            
indices, patches = zip(*sliding_window(test_image))
patches_hog = np.array([feature.hog(patch) for patch in patches])
patches_hog.shape
#---運算每個cell的特徵值---

#---框框---
#框出與訓練集人臉資料一樣的特徵
labels = model.predict(patches_hog) #預測
labels.sum()


fig, ax = plt.subplots()
ax.imshow(test_image, cmap='gray') #以灰階顯示圖片
ax.axis('off')

Ni, Nj = positive_patches[0].shape
indices = np.array(indices)

for i, j in indices[labels == 1]:
    ax.add_patch(plt.Rectangle((j, i), Nj, Ni, edgecolor='red', alpha=0.3, lw=2, facecolor='none'))
#---框框---

'wget' 不是內部或外部命令、可執行的程式或批次檔。


ValueError: negative dimensions are not allowed